In [ ]:
from gdsfactory.cross_section import ComponentAlongPath



    # Drive line
    drive_xmon = gf.read.import_gds(gdspath='drive-xmon.gds')
    drive_xmon.add_port('back', center=[drive_xmon.dxmin,(drive_xmon.dymax + drive_xmon.dymin)/2], layer=(5,0), width=10, orientation=180)
    drive_xmon.add_port('front', center=[drive_xmon.dxmax,(drive_xmon.dymax + drive_xmon.dymin)/2], layer = (5,0), width=10, orientation=180)
    drive_xmon_ref = canvas_qubit << drive_xmon
    drive_xmon_ref.connect("front", xmon_ref.ports['drive'], allow_layer_mismatch=True, allow_width_mismatch=True)

    inner = gf.Component()
    outer = gf.Component()
    bridge = gf.Component()
    xs_1 = gf.cross_section.cross_section(width=tranmission_width_drive, layer=(5,0))
    print(type(gf.c.rectangle()))
    via = ComponentAlongPath(
        component=gf.c.rectangle(size=(30, 30), centered=True, layer=(30,0)), spacing=100, padding=2, offset=10,
    )
    xs_2 = gf.cross_section.cross_section(width=tranmission_width_drive + 2*tranmission_tunnel_width_drive, layer=(5,0))

    xs_3_Section = gf.Section(width=tranmission_tunnel_width_drive+2*tranmission_tunnel_width_drive, layer=(5,0), port_names=('o1', 'o2'))
    xs_3 = gf.CrossSection(sections=[xs_3_Section], components_along_path=[via])


    route_inner = gf.routing.route_single(
        inner, 
        port1 = left_pad.ports['front'],
        port2 = drive_xmon_ref.ports['back'],
        allow_width_mismatch = True,
        cross_section = xs_1,
        radius = route_radius,
    )

    route_outer = gf.routing.route_single(
        outer, 
        port1 = left_pad.ports['front'],
        port2 = drive_xmon_ref.ports['back'],
        allow_width_mismatch = True,
        cross_section = xs_2,
        radius = route_radius,
    )
    route_bridge = gf.routing.route_single(
        bridge, 
        port1 = left_pad.ports['front'],
        port2 = drive_xmon_ref.ports['back'],
        allow_width_mismatch = True,
        cross_section = xs_3,
        radius = route_radius,

    )



    drive_tunnel = gf.boolean(A=outer, B = inner, operation='A-B', layer=(5,0))
    drive_tunnel_ref = canvas_qubit << drive_tunnel

    extracted = bridge.extract(layers=((30, 0),))
    extracted.name = 'extracted'
    extracted.show()

